## پروژه اول درس هوش مصنوعی با موضوع الگوریتم های جستجوی آگاهانه و ناآگاهانه

### در این پروژه سعی میکنیم با استفاده از الگوریتم های زیر مساله ی یافتن راهی برای جمع آوری معجون ها از معبدی با شکل شطرنجی را حل کنیم

## BFS , IDS & A*

# Imports

In [302]:
import numpy as np
import time
from copy import deepcopy
import queue as qu

# Body

## Load tests data

In [57]:
tests = []
file = open("test1.in")
tests.append(file.readlines())
file.close()
file = open("test2.in")
tests.append(file.readlines())
file.close()
file = open("test3.in")
tests.append(file.readlines())
file.close()

#### داده های تست ها را از فایل هایی که در پوشه پروژه قرار دارند برای تست برنامه لود می کنیم
#### با توجه به تست های موجود فرض می شود که تعداد سطر ها و ستون ها اعداد تک رقمی می باشند 

## Create tests maps

In [58]:
maps = []
Cs = []
Ks = []
for test in tests:
    num_of_row = int(test[0][0])
    num_of_col = int(test[0][2])
    curr_map = [['e'] * num_of_col for i in range(num_of_row)]
    curr_c = int(test[1][0])
    curr_k = int(test[1][2])
    curr_Cs = []
    curr_Ks = []
    for i in range(0, curr_c):
        curr_Cs.append([int(test[2+i][0]), int(test[2+i][2]), False])
        curr_map[int(test[2+i][0])][int(test[2+i][2])] = 'c'
    for i in range(0, curr_k):
        curr_Ks.append([int(test[2+curr_c+i][0]), int(test[2+curr_c+i][2]), False])
        curr_map[int(test[2+curr_c+i][0])][int(test[2+curr_c+i][2])] = 'k'
    curr_d = int(test[2+curr_c+curr_k])
    for i in range(0, curr_d):
        curr_map[int(test[3+curr_c+curr_k+i][0])][int(test[3+curr_c+curr_k+i][2])] = 'd'
    maps.append(curr_map)
    Cs.append(curr_Cs)
    Ks.append(curr_Ks)


In [59]:
maps

[[['e', 'k', 'c', 'e'],
  ['e', 'e', 'd', 'e'],
  ['c', 'e', 'e', 'e'],
  ['e', 'e', 'e', 'e']],
 [['k', 'e', 'e', 'd'],
  ['e', 'k', 'c', 'd'],
  ['c', 'c', 'e', 'e'],
  ['e', 'k', 'e', 'e']],
 [['e', 'e', 'd', 'e', 'e', 'e'],
  ['e', 'd', 'c', 'e', 'e', 'c'],
  ['e', 'e', 'k', 'e', 'e', 'e'],
  ['d', 'e', 'e', 'e', 'e', 'c'],
  ['e', 'd', 'k', 'c', 'e', 'e'],
  ['e', 'c', 'e', 'e', 'e', 'e']]]

#### مشاهده می شود که نقشه های تست به درستی  لود شده اند

## Model Description

### Actions : Di up, Di down, Di right, Di left (for i in range num of doctors)

### Initial State : A doctor in (0, 0) and the map looks like the test files

### Goal State : Every doctor in (n-1, m-1) and no 'c' remain in the map

### Path cost : sum of moves (Actions) of all doctors (from Initial state to Goal state)

### State : position of elements in the map in a node represents the current state for this node

### Transition Model : اگر دکتری به خانه ای که دارای دارو است وارد شود آن خانه تبدیل به خانه ی خالی شده و یک دکتر در خانه ی پایین سمت چپ جدول ایجاد می شود . اگر دکتری وارد خانه ای دارای معجون شود آن خانه به خانه ی خالی تبدیل می شود . برای حرکت دکتر ها هم تنها در صورتی که خانه ی مورد نظر مجاز باشد و قابل رفتن باشد به آنجا می رود

## Common Functions

In [326]:
def is_in_expanded(node, expanded):
    #result = np.where(node.is_same(expanded), True, False)
    #if True in result:
    #    return True
    #return False
    ##############
    for nd in expanded:
        if nd.is_same(node):
            return True
    return False
    ############
    #node_list = [node]*len(expanded)
    #result = map(is_same, node_list, expanded)
    #if True in result:
    #    return True
    #return False
    
def eat_drug(Ks, x, y):
    for k in Ks:
        if k[0] == x and k[1] == y:
            if k[2]:
                return False
            else:
                k[2] = True
                return True
            
def pick_confection(Cs, x, y):
    for c in Cs:
        if c[0] == x and c[1] == y:
            c[2] = True
            return
        
def is_dr_in_list(dr, drs):
    for d in drs:
        if d[1] == dr[1] and d[2] == dr[2]:
            return True
    return False

## Data Structures

In [327]:
class Node:
    def __init__(self, curr_map, curr_Cs, curr_Ks, path_until_there, path_cost_until_there, curr_Drs):
        self.map = curr_map
        self.Cs = curr_Cs
        self.Ks = curr_Ks
        self.path = path_until_there
        self.path_cost = path_cost_until_there
        self.Drs = curr_Drs
        self.m = len(curr_map)
        self.n = len(curr_map[0])
        if self.path == 0:
            if self.map[0][0] == 'k':
                eat_drug(self.Ks, 0, 0)
                self.Drs.append([1, self.m-1, 0])
    
    def Goal_test(self):
        for c in self.Cs:
            if c[2] is False:
                return False
        for dr in self.Drs:
            if dr[1] != self.m-1 or dr[2] != self.n-1:
                return False
        return True
    
    def expand(self, fifo, expanded):
        ##if expanded:
        ##    if expanded[-1].path_cost < self.path_cost:
        ##        expanded = []
        expanded.append(self)
        for dr in self.Drs:
            if dr[1] == self.m-1 and dr[2] == self.n-1:
                continue
            if dr[1]+1 < self.m:
                if self.map[dr[1]+1][dr[2]] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]+1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]+1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]+1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]+1, dr[2])
                    #temp_state[dr[1]+1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "D"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs)
                    if not is_in_expanded(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        fifo.put(temp_node)
            if dr[1] > 0:
                if self.map[dr[1]-1][dr[2]] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][1] = dr[1]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]-1][dr[2]] == 'k':
                        if eat_drug(temp_Ks, dr[1]-1, dr[2]):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]-1][dr[2]] == 'c':
                        pick_confection(temp_Cs, dr[1]-1, dr[2])
                    #temp_state[dr[1]-1][dr[2]] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "U"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs)
                    if not is_in_expanded(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        fifo.put(temp_node)
            if dr[2]+1 < self.n:
                if self.map[dr[1]][dr[2]+1] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]+1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]+1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]+1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]+1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]+1)
                    #temp_state[dr[1]][dr[2]+1] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "R"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs)
                    if not is_in_expanded(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        fifo.put(temp_node)
            if dr[2] > 0:
                if self.map[dr[1]][dr[2]-1] != 'd':
                    #temp_state = deepcopy(self.state)
                    temp_drs = deepcopy(self.Drs)
                    temp_drs[dr[0]][2] = dr[2]-1
                    temp_Ks = deepcopy(self.Ks)
                    temp_Cs = deepcopy(self.Cs)
                    if self.map[dr[1]][dr[2]-1] == 'k':
                        if eat_drug(temp_Ks, dr[1], dr[2]-1):
                            temp_drs.append([len(self.Drs), self.m-1, 0])
                    if self.map[dr[1]][dr[2]-1] == 'c':
                        pick_confection(temp_Cs, dr[1], dr[2]-1)
                    #temp_state[dr[1]][dr[2]-1] = 'e'
                    temp_path = deepcopy(self.path)
                    temp_path.append([dr[0], "L"])
                    temp_node = Node(self.map, temp_Cs, temp_Ks, temp_path, self.path_cost+1, temp_drs)
                    if not is_in_expanded(temp_node, expanded):
                        if temp_node.Goal_test():
                            return temp_node
                        fifo.put(temp_node)
                    
        return None
    
    def is_same(self, node):
        #if len(self.Drs) == len(node.Drs):
        if self.Cs == node.Cs and self.Ks == node.Ks:
            for dr in self.Drs:
                if not is_dr_in_list(dr, node.Drs):
                    return False
            return True
        return False

## Implementation with BFS

### این الگوریتم کامل است و همواره جواب بهینه را به دست می آورد ولی از نظر تعداد استیت های دیده شده و زمان اجرا بهینه نیست 

## My Test

In [328]:
maps_copy = deepcopy(maps)

In [329]:
Cs_copy = deepcopy(Cs)

In [330]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['c', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [331]:
Cs_copy[0] = [[0, 2, False]]

In [332]:
maps_copy[0][2][0] = 'e'

In [333]:
maps_copy[0]

[['e', 'k', 'c', 'e'],
 ['e', 'e', 'd', 'e'],
 ['e', 'e', 'e', 'e'],
 ['e', 'e', 'e', 'e']]

In [334]:
fifo = qu.Queue()
expanded = []

In [335]:
fifo.put(Node(maps_copy[0], Cs_copy[0], Ks[0], [], 0, [[0, 0, 0]]))

In [336]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 9
[[0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'R'], [1, 'R'], [1, 'R']]


In [337]:
bfs_time = last_time - first_time

In [338]:
bfs_time

1.4813413619995117

In [339]:
len(expanded)

1957

## Test 1

In [340]:
fifo = qu.Queue()
expanded = []

In [341]:
fifo.put(Node(maps[0], Cs[0], Ks[0], [], 0, [[0, 0, 0]]))

In [342]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 11
[[0, 'R'], [0, 'R'], [0, 'R'], [0, 'D'], [0, 'D'], [0, 'D'], [1, 'U'], [1, 'D'], [1, 'R'], [1, 'R'], [1, 'R']]


In [343]:
bfs_time = last_time - first_time

In [344]:
bfs_time

11.332806587219238

In [345]:
len(expanded)

6009

## Test 2

In [346]:
fifo = qu.Queue()
expanded = []

In [347]:
fifo.put(Node(maps[1], Cs[1], Ks[1], [], 0, [[0, 0, 0]]))

In [348]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

answer found
path cost = 8
[[0, 'D'], [0, 'D'], [0, 'R'], [0, 'R'], [0, 'U'], [0, 'D'], [0, 'D'], [0, 'R']]


In [349]:
bfs_time = last_time - first_time

In [350]:
bfs_time

14.015931129455566

In [351]:
len(expanded)

4552

## Test 3

In [245]:
fifo = qu.Queue()
expanded = []

In [246]:
fifo.put(Node(maps[2], Cs[2], Ks[2], [], 0, [[0, 0, 0]]))

In [247]:
first_time = time.time()
while not fifo.empty():
    curr_node = fifo.get()
    answer = curr_node.expand(fifo, expanded)
    if answer != None:
        print("answer found")
        print("path cost =", answer.path_cost)
        print(answer.path)
        break
last_time = time.time()

KeyboardInterrupt: 

In [ ]:
bfs_time = last_time - first_time

In [ ]:
bfs_time

In [ ]:
len(expanded)

## Implementation with IDS

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [185]:
maps_copy = deepcopy(maps)

## Implementation with A* with consistent heuristic

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [186]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.3 

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [187]:
maps_copy = deepcopy(maps)

## Implementation with weighted A* with Alfa = 1.5

### این الگوریتم کامل .......... و همواره جواب بهینه را .............. ولی از نظر تعداد استیت ها و زمان اجرا بهینه نیست 

In [188]:
maps_copy = deepcopy(maps)